# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal business idea. Specifically, this report will be targeted to stakeholders interested in opening business based on the **most visits** in New York City, USA.

Since the area of NYC is really big, therefore, we would focus on Staten Island and its most visited venues in each neighbourhood.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible business category can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing businesses in the neighborhood (any type of business)

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* I will be using the New York City data provided in the previous weeks to generate a map as lon and lat are already present there.

* number of businesses and their types and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of NYC will be obtained using **Google Maps API geocoding** 

### Before we move further, lets first download all the libraries we need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download the Dataset


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

Lets have a peek at the data!!

In [5]:
neighborhoods_data

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

So far so good. Now we transform our data into pandas dataframe.

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

Our empty dataframe looks like this:

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Now we loop through the data and fill the dataframe one row at a time.


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Now we make sure our dataset has 5 boroughs and 306 neighborhoods.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Bingo!!

#### Now we use the geopy library to get the coordinates of New York City. Once we have the coordinates of the city, we will plot the map with the neighborhoods.

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Now that we have the map of NYC plotted, we notice that its too big and clustered. For this we select the borough of our choice to make it less complicated. We can select any borough but since my in laws reside in Staten Island, i will explore SI.

In [13]:
staten_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
staten_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


Now we perform the same procedure again to get the coordinates of Staten Island and then plot the map with the neighborhoods on it.

In [14]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Staten Island are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Staten Island are 40.5834557, -74.1496048.


In [15]:
map_staten = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(staten_data['Latitude'], staten_data['Longitude'], staten_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_staten)  
    
map_staten

Done and dusted. Now comes Foursquare API to explore the neighborhoods.

In [17]:
CLIENT_ID = 'ID44JB0UA0QQWNDHHJI4YDIC3P31R0QEGG1SINF3A41L1BTM' 
CLIENT_SECRET = 'LTK1SAVZSIR1OVICIECE52YYCTWXIGMI41OWLEAJEIXF1I4X'
VERSION = '20180605'
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ID44JB0UA0QQWNDHHJI4YDIC3P31R0QEGG1SINF3A41L1BTM
CLIENT_SECRET:LTK1SAVZSIR1OVICIECE52YYCTWXIGMI41OWLEAJEIXF1I4X


Now we explore the first neighborhood in SI and find out the coordinates for it.

In [18]:
staten_data.loc[0, 'Neighborhood']

'St. George'

In [19]:
neighborhood_latitude = staten_data.loc[0, 'Latitude'] 
neighborhood_longitude = staten_data.loc[0, 'Longitude']

neighborhood_name = staten_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. George are 40.6449815710044, -74.07935312512797.


Now we get the top 100 venues of St. George

In [20]:
LIMIT = 100 
radius = 500
 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=ID44JB0UA0QQWNDHHJI4YDIC3P31R0QEGG1SINF3A41L1BTM&client_secret=LTK1SAVZSIR1OVICIECE52YYCTWXIGMI41OWLEAJEIXF1I4X&v=20180605&ll=40.6449815710044,-74.07935312512797&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d871ee843c1720d0fb708b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 40.6494815755044,
    'lng': -74.07343346476772},
   'sw': {'lat': 40.6404815665044, 'lng': -74.08527278548821}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6924c46d4a5938e7ac6c0',
       'name': 'A&S Pizzeria',
       'location': {'address': '87 Stuyvesant Pl',
        'crossStreet': 'Wall st',
        'lat': 40.64393953223924,
        'lng': -74.0776259226109,
 

Now we define a function to get the category type of each venue.

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-23-a608756be7e4>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


,name,categories,lat,lng
0,A&S Pizzeria,Pizza Place,40.643940,-74.077626
1,Beso,Tapas Restaurant,40.643306,-74.076508
2,Shake Shack,Burger Joint,40.643660,-74.075891
3,The Gavel Grill,American Restaurant,40.642157,-74.076674
4,Staten Island September 11 Memorial,Monument / Landmark,40.646767,-74.076510


Now we know how to get venue categories, so we explore all the neighborhoods and their most popular venues.

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we run our code and create a new dataframe for all the venues.

In [25]:
staten_venues = getNearbyVenues(names=staten_data['Neighborhood'],
                                   latitudes=staten_data['Latitude'],
                                   longitudes=staten_data['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [29]:
print(staten_venues.shape)
staten_venues.head()

(791, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,40.644982,-74.079353,Shake Shack,40.643660,-74.075891,Burger Joint
3,St. George,40.644982,-74.079353,The Gavel Grill,40.642157,-74.076674,American Restaurant
4,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark


In [30]:
staten_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,13,13,13,13,13,13
Arden Heights,6,6,6,6,6,6
Arlington,4,4,4,4,4,4
Arrochar,19,19,19,19,19,19
Bay Terrace,12,12,12,12,12,12
Bloomfield,4,4,4,4,4,4
Bulls Head,43,43,43,43,43,43
Butler Manor,5,5,5,5,5,5
Castleton Corners,14,14,14,14,14,14


In [31]:
staten_onehot = pd.get_dummies(staten_venues[['Venue Category']], prefix="", prefix_sep="")

staten_onehot['Neighborhood'] = staten_venues['Neighborhood'] 

fixed_columns = [staten_onehot.columns[-1]] + list(staten_onehot.columns[:-1])
staten_onehot = staten_onehot[fixed_columns]

staten_onehot.head()

,Yoga Studio,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Event Service,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Laundromat,Lawyer,Liquor Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightlife Spot,Optical Shop,Outdoors & Recreation,Outlet Mall,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Professional & Other Places,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
staten_onehot.shape

(791, 175)

In [33]:
staten_grouped = staten_onehot.groupby('Neighborhood').mean().reset_index()
staten_grouped

,Neighborhood,Yoga Studio,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Event Service,Event Space,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Laundromat,Lawyer,Liquor Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightlife Spot,Optical Shop,Outdoors & Recreation,Outlet Mall,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Professional & Other Places,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Annadale,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.076923,0.000,0.000000,0.076923,0.00,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.00000,0.000000,0.076923,0.000000,0.000000,0.000000

In [34]:
staten_grouped.shape

(62, 175)

Now we print all the neighborhoods with their top 5 venues.

In [35]:
num_top_venues = 5

for hood in staten_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = staten_grouped[staten_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Annadale----
          venue  freq
0   Pizza Place  0.15
1  Liquor Store  0.08
2        Bakery  0.08
3         Diner  0.08
4  Dance Studio  0.08


----Arden Heights----
           venue  freq
0  Deli / Bodega  0.17
1   Home Service  0.17
2       Bus Stop  0.17
3       Pharmacy  0.17
4    Coffee Shop  0.17


----Arlington----
           venue  freq
0  Deli / Bodega  0.25
1  Boat or Ferry  0.25
2       Bus Stop  0.25
3  Grocery Store  0.25
4     Playground  0.00


----Arrochar----
                   venue  freq
0            Pizza Place  0.11
1               Bus Stop  0.11
2     Italian Restaurant  0.11
3  Outdoors & Recreation  0.05
4             Nail Salon  0.05


----Bay Terrace----
                venue  freq
0         Supermarket  0.17
1  Italian Restaurant  0.17
2      Shipping Store  0.08
3          Donut Shop  0.08
4       Train Station  0.08


----Bloomfield----
               venue  freq
0  Recreation Center  0.25
1       Burger Joint  0.25
2           Bus Stop  0.25
3      

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = staten_grouped['Neighborhood']

for ind in np.arange(staten_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(staten_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Pizza Place,Diner,Sushi Restaurant,Park,Pharmacy,Bakery,Liquor Store,Dance Studio,Restaurant,American Restaurant
1,Arden Heights,Deli / Bodega,Coffee Shop,Bus Stop,Home Service,Pharmacy,Pizza Place,Eastern European Restaurant,Flower Shop,Filipino Restaurant,Fast Food Restaurant
2,Arlington,Deli / Bodega,Boat or Ferry,Bus Stop,Grocery Store,Event Space,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
3,Arrochar,Pizza Place,Italian Restaurant,Bus Stop,Pharmacy,Supermarket,Mediterranean Restaurant,Outdoors & Recreation,Nail Salon,Sandwich Place,Food Truck
4,Bay Terrace,Italian Restaurant,Supermarket,Donut Shop,Insurance Office,Train Station,Plaza,Salon / Barbershop,Sushi Restaurant,Grocery Store,Shipping Store


Now we have our dataset. Each neighborhood has the most common venues(10 venues). Furthermore, we split our data set into clusters to better define them.

In [38]:
kclusters = 5

staten_grouped_clustering = staten_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(staten_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 3, 2, 3, 3, 2, 3, 3, 3, 3], dtype=int32)

Now we merge our data set to append the coordinates in it.

In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

staten_merged = staten_data

staten_merged = staten_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

staten_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,3.0,Clothing Store,Sporting Goods Shop,American Restaurant,Bar,Donut Shop,Pizza Place,Monument / Landmark,Burger Joint,Bus Station,Scenic Lookout
1,Staten Island,New Brighton,40.640615,-74.087017,2.0,Bus Stop,Deli / Bodega,Park,Discount Store,Playground,Bowling Alley,Laundromat,Diner,Falafel Restaurant,Food
2,Staten Island,Stapleton,40.626928,-74.077902,3.0,Coffee Shop,Bank,Pizza Place,Discount Store,Sandwich Place,Breakfast Spot,Spanish Restaurant,Fast Food Restaurant,Motorcycle Shop,Skate Park
3,Staten Island,Rosebank,40.615305,-74.069805,3.0,Italian Restaurant,Mexican Restaurant,Filipino Restaurant,Martial Arts School,Mobile Phone Shop,Storage Facility,Sandwich Place,Chinese Restaurant,Donut Shop,Beach
4,Staten Island,West Brighton,40.631879,-74.107182,3.0,Coffee Shop,Italian Restaurant,Sandwich Place,Bank,Ice Cream Shop,Salon / Barbershop,German Restaurant,Breakfast Spot,Burger Joint,Bus Stop


Now we drop all the columns and rows with null values in it. And convert cluster values to integers.

In [50]:
staten_merged.dropna(subset = ['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue'], thresh=1)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,3.0,Clothing Store,Sporting Goods Shop,American Restaurant,Bar,Donut Shop,Pizza Place,Monument / Landmark,Burger Joint,Bus Station,Scenic Lookout
1,Staten Island,New Brighton,40.640615,-74.087017,2.0,Bus Stop,Deli / Bodega,Park,Discount Store,Playground,Bowling Alley,Laundromat,Diner,Falafel Restaurant,Food
2,Staten Island,Stapleton,40.626928,-74.077902,3.0,Coffee Shop,Bank,Pizza Place,Discount Store,Sandwich Place,Breakfast Spot,Spanish Restaurant,Fast Food Restaurant,Motorcycle Shop,Skate Park
3,Staten Island,Rosebank,40.615305,-74.069805,3.0,Italian Restaurant,Mexican Restaurant,Filipino Restaurant,Martial Arts School,Mobile Phone Shop,Storage Facility,Sandwich Place,Chinese Restaurant,Donut Shop,Beach
4,Staten Island,West Brighton,40.631879,-74.107182,3.0,Coffee Shop,Italian Restaurant,Sandwich Place,Bank,Ice Cream Shop,Salon / Barbershop,German Restaurant,Breakfast Spot,Burger Joint,Bus Stop
5,Staten Island,Grymes Hill,40.624185,-74.087248,2.0,Bus Stop,Dog Run,Women's Store,Event Service,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
6,Staten Island,Todt Hill,40.597069,-74.111329,1.0,Park,Trail,Women's Store,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
7,Staten Island,South Beach,40.580247,-74.079553,3.0,Beach,Pier,Theme Park,Deli / Bodega,Bus Stop,Athletics & Sports,Event Service,Food,Flower Shop,Filipino Restaurant
8,Staten Island,Port Richmond,40.633669,-74.129434,3.0,Rental Car Location,Sports Bar,Park,Martial Arts School,Basketball Court,Eastern European Restaurant,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
9,Staten Island,Mariner's Harbor,40.632546,-74.150085,2.0,Deli / Bodega,Supermarket,Bus Stop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [51]:
staten_merged.fillna(0)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,3.0,Clothing Store,Sporting Goods Shop,American Restaurant,Bar,Donut Shop,Pizza Place,Monument / Landmark,Burger Joint,Bus Station,Scenic Lookout
1,Staten Island,New Brighton,40.640615,-74.087017,2.0,Bus Stop,Deli / Bodega,Park,Discount Store,Playground,Bowling Alley,Laundromat,Diner,Falafel Restaurant,Food
2,Staten Island,Stapleton,40.626928,-74.077902,3.0,Coffee Shop,Bank,Pizza Place,Discount Store,Sandwich Place,Breakfast Spot,Spanish Restaurant,Fast Food Restaurant,Motorcycle Shop,Skate Park
3,Staten Island,Rosebank,40.615305,-74.069805,3.0,Italian Restaurant,Mexican Restaurant,Filipino Restaurant,Martial Arts School,Mobile Phone Shop,Storage Facility,Sandwich Place,Chinese Restaurant,Donut Shop,Beach
4,Staten Island,West Brighton,40.631879,-74.107182,3.0,Coffee Shop,Italian Restaurant,Sandwich Place,Bank,Ice Cream Shop,Salon / Barbershop,German Restaurant,Breakfast Spot,Burger Joint,Bus Stop
5,Staten Island,Grymes Hill,40.624185,-74.087248,2.0,Bus Stop,Dog Run,Women's Store,Event Service,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
6,Staten Island,Todt Hill,40.597069,-74.111329,1.0,Park,Trail,Women's Store,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
7,Staten Island,South Beach,40.580247,-74.079553,3.0,Beach,Pier,Theme Park,Deli / Bodega,Bus Stop,Athletics & Sports,Event Service,Food,Flower Shop,Filipino Restaurant
8,Staten Island,Port Richmond,40.633669,-74.129434,3.0,Rental Car Location,Sports Bar,Park,Martial Arts School,Basketball Court,Eastern European Restaurant,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
9,Staten Island,Mariner's Harbor,40.632546,-74.150085,2.0,Deli / Bodega,Supermarket,Bus Stop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(staten_merged['Latitude'], staten_merged['Longitude'], staten_merged['Neighborhood'], staten_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Methodology

In this project we will direct our efforts on detecting what kind of businesses or attractions are most popular among visitors. 

In first step we have collected the required **data: location and type (category) of every attraction/venue Staten Island**. 

Second and final step in our analysis will be exploration and categorization of these venues **' across different neighborhoods of Staten Island - we will use **clustering** to categorize the venues to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

**Cluster 1**

In [59]:
staten_merged.loc[staten_merged['Cluster Labels'] == 0, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Richmond Town,Italian Restaurant,Spa,Bagel Shop,Café,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
60,Lighthouse Hill,Italian Restaurant,Art Museum,Spa,Trail,Massage Studio,Café,Women's Store,Eastern European Restaurant,Flower Shop,Filipino Restaurant


**Cluster 2**

In [60]:
staten_merged.loc[staten_merged['Cluster Labels'] == 1, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Todt Hill,Park,Trail,Women's Store,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space


**Cluster 3**

In [61]:
staten_merged.loc[staten_merged['Cluster Labels'] == 2, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,New Brighton,Bus Stop,Deli / Bodega,Park,Discount Store,Playground,Bowling Alley,Laundromat,Diner,Falafel Restaurant,Food
5,Grymes Hill,Bus Stop,Dog Run,Women's Store,Event Service,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
9,Mariner's Harbor,Deli / Bodega,Supermarket,Bus Stop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
24,Park Hill,Bus Stop,Athletics & Sports,Coffee Shop,Hotel,Women's Store,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
26,Graniteville,Food Truck,Boat or Ferry,Bus Stop,Grocery Store,Event Space,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
27,Arlington,Deli / Bodega,Boat or Ferry,Bus Stop,Grocery Store,Event Space,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
44,Chelsea,Bus Stop,Steakhouse,Theater,Arts & Crafts Store,Sandwich Place,Park,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
45,Bloomfield,Recreation Center,Theme Park,Burger Joint,Bus Stop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
56,Willowbrook,Bus Stop,Deli / Bodega,Intersection,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Event Space
57,Sandy Ground,Bus Stop,Intersection,Home Service,Women's Store,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


**Cluster 4**

In [62]:
staten_merged.loc[staten_merged['Cluster Labels'] == 3, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. George,Clothing Store,Sporting Goods Shop,American Restaurant,Bar,Donut Shop,Pizza Place,Monument / Landmark,Burger Joint,Bus Station,Scenic Lookout
2,Stapleton,Coffee Shop,Bank,Pizza Place,Discount Store,Sandwich Place,Breakfast Spot,Spanish Restaurant,Fast Food Restaurant,Motorcycle Shop,Skate Park
3,Rosebank,Italian Restaurant,Mexican Restaurant,Filipino Restaurant,Martial Arts School,Mobile Phone Shop,Storage Facility,Sandwich Place,Chinese Restaurant,Donut Shop,Beach
4,West Brighton,Coffee Shop,Italian Restaurant,Sandwich Place,Bank,Ice Cream Shop,Salon / Barbershop,German Restaurant,Breakfast Spot,Burger Joint,Bus Stop
7,South Beach,Beach,Pier,Theme Park,Deli / Bodega,Bus Stop,Athletics & Sports,Event Service,Food,Flower Shop,Filipino Restaurant
8,Port Richmond,Rental Car Location,Sports Bar,Park,Martial Arts School,Basketball Court,Eastern European Restaurant,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
11,Castleton Corners,Pizza Place,Thai Restaurant,Sandwich Place,Tattoo Parlor,Grocery Store,Burger Joint,Bar,Bank,Bagel Shop,Hardware Store
12,New Springville,Mobile Phone Shop,Chinese Restaurant,Flower Shop,Liquor Store,Optical Shop,Shopping Mall,Donut Shop,Sandwich Place,Mexican Restaurant,Soup Place
13,Travis,Hotel,Gym,Bowling Alley,Baseball Field,Donut Shop,Food,Deli / Bodega,Park,Sports Club,Comedy Club
14,New Dorp,Italian Restaurant,Pizza Place,Yoga Studio,Taco Place,Gas Station,Mexican Restaurant,Food,Dim Sum Restaurant,Dessert Shop,Deli / Bodega


**Cluster 5**

In [63]:
staten_merged.loc[staten_merged['Cluster Labels'] == 4, staten_merged.columns[[1] + list(range(5, staten_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Port Ivory,Business Service,Bar,Women's Store,French Restaurant,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
15,Oakwood,Nightlife Spot,Lawyer,Home Service,Bar,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
22,Silver Lake,American Restaurant,Burger Joint,Museum,Golf Course,Women's Store,Event Service,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant
58,Egbertville,Cosmetics Shop,Italian Restaurant,Clothing Store,Bagel Shop,Home Service,Eastern European Restaurant,Food,Flower Shop,Filipino Restaurant,Fast Food Restaurant


### Result

From our analysis we have categorized the venues in 5 clusters which shows Top 10 venues in every venue.

### Conclusion

Purpose of this project was for stakeholders/entrepreneurs to identify the top venues in every neighborhoods which can help them in choosing the best options for their business interests.

Final decision on optimal business idea will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.